In [35]:
import sys
import torch
import dgl
import numpy as np
from sklearn.neighbors import NearestNeighbors
import recommend as r
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [36]:
sys.path.insert(0,'/home/adarsh/projects/new-folder/utils')
from utils.utils import load_features, load_graph

In [74]:
# Load graph, feat_data
feat_dir = "/home/adarsh/projects/new-folder/data/songset_features.csv"
feat_data, uri_map = load_features(feat_dir)

graph_dir = "/home/adarsh/projects/new-folder/data/graph_main.gpickle"
dgl_G, weights = load_graph(graph_dir, uri_map)

Loading feature data...
Feature data shape: torch.Size([34443, 13])
Loading graph data...
Graph Info:
 Graph named 'G' with 34443 nodes and 3377538 edges


In [75]:
print(len(uri_map))

34443


In [76]:
import json
def load_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

file_path = '/home/adarsh/projects/new-folder/model/uri_map_170k.json'
uri_map = load_json_file(file_path)

In [77]:
print(len(uri_map))

170089


In [38]:
from dgl.nn import SAGEConv
import torch.nn as nn
import torch.nn.functional as F

class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat, weights):
        h = self.conv1(g, in_feat, edge_weight=weights[g.edata[dgl.EID]])
        h = F.relu(h)
        h = self.conv2(g, h, edge_weight=weights[g.edata[dgl.EID]])
        return h
    
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}
    
    def forward(self, g, h, edges='__ALL__'):
        with g.local_scope():
            g.ndata['h'] = h
            if edges != '__ALL__':
                g.apply_edges(self.apply_edges, edges=edges)
            else: 
                g.apply_edges(self.apply_edges)
            return g.edata['score']

In [39]:
model = GraphSAGE(13,10)
pred = MLPPredictor(10)
model_path = '/home/adarsh/projects/new-folder/model/460k_model.pt'
pred_path = '/home/adarsh/projects/new-folder/model/460k_pred.pt'

In [40]:
# Load Pre-Existing Model
model = torch.load(model_path)
pred = torch.load(pred_path)
model.eval()
pred.eval()

MLPPredictor(
  (W1): Linear(in_features=20, out_features=10, bias=True)
  (W2): Linear(in_features=10, out_features=1, bias=True)
)

In [42]:
dgl_G.edata[dgl.EID] = torch.tensor(np.arange(dgl_G.number_of_edges()))

In [45]:
feats_splitted = np.array_split(np.arange(34443), 500)

In [46]:
preds = torch.tensor([])
z = torch.tensor([])
for i in feats_splitted:
    temp_graph = dgl.node_subgraph(dgl_G, i)
    
    # Create Embeddings
    z_i = model(temp_graph, feat_data[i], weights)

    # Create Predictions
    # In the form of the strength of the connection between source, destination from dgl_G.edges()
    pred_i = pred(temp_graph, z_i)
    preds = torch.cat((preds, pred_i), 0)
    z = torch.cat((z, z_i), 0)

In [47]:
print(preds.shape)
print(z.shape)

torch.Size([13364])
torch.Size([34443, 10])


In [48]:
neigh = NearestNeighbors(n_neighbors=25, radius=0.4)
neigh.fit(feat_data)

NearestNeighbors(n_neighbors=25, radius=0.4)

In [53]:
from spotipy.oauth2 import SpotifyClientCredentials
# Add cilent_id and client_secret
client_id = None
client_secret = None
redirect_uri = "https://localhost:8080/"
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id,client_secret,redirect_uri))

results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " - ", track['name'])


HTTP Error for GET to https://api.spotify.com/v1/me/tracks with Params: {'limit': 20, 'offset': 0, 'market': None} returned 403 due to Forbidden.


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/me/tracks?limit=20&offset=0:
 Forbidden., reason: None

In [78]:
item = {'name': 'Happy :)',
 'collaborative': 'false',
 'pid': 9360,
 'modified_at': 1500854400,
 'num_tracks': 3,
 'num_albums': 10,
 'num_followers': 2,
 'tracks': [{'pos': 0,
   'artist_name': 'Phoenix',
   'track_uri': 'spotify:track:6ZrlXT6mUMEgomN128iekU',
   'artist_uri': 'spotify:artist:1xU878Z1QtBldR7ru9owdU',
   'track_name': 'Lisztomania',
   'album_uri': 'spotify:album:6YXmQrXOjJoMheJ2IA5NqK',
   'duration_ms': 241640,
   'album_name': 'Wolfgang Amadeus Phoenix'},
  {'pos': 1,
   'artist_name': 'Michael Jackson',
   'track_uri': 'spotify:track:2rN2WzftSKZzDLFApWECWz',
   'artist_uri': 'spotify:artist:3fMbdgg4jU18AjLCKBhRSm',
   'track_name': 'Will You Be There (Theme from "Free Willy")',
   'album_uri': 'spotify:album:48TFZhC2UmuzNL23WshdwW',
   'duration_ms': 352893,
   'album_name': 'FREE WILLY - ORIGINAL MOTION PICTURE SOUNDTRACK'},
  {'pos': 2,
   'artist_name': 'Lake Street Dive',
   'track_uri': 'spotify:track:6FxKh5EdB8S22V4i3BORsr',
   'artist_uri': 'spotify:artist:3nuc29fYGlQbIrwh4yrNWd',
   'track_name': 'Rabid Animal',
   'album_uri': 'spotify:album:2AxRR7XLqFFJuMtjn9fpOF',
   'duration_ms': 133779,
   'album_name': 'Bad Self Portraits'}],
 'num_edits': 10,
 'duration_ms': 2315002,
 'num_artists': 10}

In [79]:
item, original_tracks, seeds = r.get_playlist_info(item)
print(original_tracks)

Playlist ID: 9360
Playlist Length: 3
['Lisztomania---Phoenix', 'Will You Be There (Theme from "Free Willy")---Michael Jackson', 'Rabid Animal---Lake Street Dive']


In [80]:
uri_recs = r.recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
print(uri_recs)

DGLError: u contains invalid node IDs